<h1>Introduction</h1>
This analysis looks at different angles of the fuel data for the vehicle spreadsheet provided for the project. It looks at fuel in the scope of transmission and vehicle types and looks at the average price for different products.

These are the different libraries that are necessary to complete the analysis.

In [635]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
from math import factorial as ft 
import plotly.express as px

This reads the data into a DataFrame and displays the information and the first five rows of the data.

In [636]:
ice_data = pd.read_csv('/Users/leahdeyoung/Desktop/GitHub/ice-games-practicum/moved_games.csv', encoding = "utf-8")

display(ice_data.head())
ice_data.info()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


This code checks for any fully duplicate rows. There appear to be none.

In [637]:
print(ice_data.duplicated().sum())

0


This is beginning of cleaning the data. I replaced all the column names with the lowercase version of the name.

In [638]:
ice_data = ice_data.rename(columns={
    'Name': 'name', 
    'Platform': 'platform',
    'Year_of_Release': 'year_of_release',
    'Genre': 'genre',
    'NA_sales': 'na_sales',
    'EU_sales': 'eu_sales',
    'JP_sales': 'jp_sales',
    'Other_sales': 'other_sales',
    'User_Score': 'user_score',
    'Critic_Score': 'critic_score',
    'Rating': 'rating'
})
print(ice_data.columns)

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


This code checks for duplicates in the name data by converting all the names to lowercase values and dropping any duplicates that also have a duplicate year of release. This is because some games have new releases in different years.

In [639]:
print(ice_data['name'].value_counts())
print(ice_data['name'].unique())
ice_data['name_lowercase'] = ice_data['name'].str.lower()
print(ice_data['name'].value_counts())
print(ice_data['name_lowercase'].unique())

ice_data = ice_data.drop_duplicates(subset=['name_lowercase', 'year_of_release']).reset_index(drop=True)



Need for Speed: Most Wanted                         12
Ratatouille                                          9
LEGO Marvel Super Heroes                             9
FIFA 14                                              9
Madden NFL 07                                        9
                                                    ..
Jewels of the Tropical Lost Island                   1
Sherlock Holmes and the Mystery of Osborne House     1
The King of Fighters '95 (CD)                        1
Megamind: Mega Team Unite                            1
Haitaka no Psychedelica                              1
Name: name, Length: 11559, dtype: int64
['Wii Sports' 'Super Mario Bros.' 'Mario Kart Wii' ...
 'Woody Woodpecker in Crazy Castle 5' 'LMA Manager 2007'
 'Haitaka no Psychedelica']
Need for Speed: Most Wanted                         12
Ratatouille                                          9
LEGO Marvel Super Heroes                             9
FIFA 14                                           

This code checks for names that are missing, then grabs the rows where the name is missing to view for analysis.

It looks like quite a bit of information is missing about this row; however, there were in fact sales in Europe and North America for this game, and the platform is valid and used for other games. Therefore, I decided to keep the row and add the string "Unknown Name" to the name field.

In [640]:
print(ice_data['name'].isna().sum())
print(ice_data.query("name.isna()"))
print(ice_data['platform'].value_counts().head(25))
ice_data['name'] = ice_data['name'].fillna('Unknown Name')
print(ice_data['name'].isna().sum())


1
    name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
589  NaN      GEN           1993.0   NaN      1.78      0.53       0.0   

     other_sales  critic_score user_score rating name_lowercase  
589         0.08           NaN        NaN    NaN            NaN  
PS2     1924
DS      1843
PS      1172
Wii     1002
PSP      965
PS3      757
GBA      689
X360     609
PC       575
3DS      451
XB       336
PS4      334
PSV      328
N64      286
SNES     237
GC       223
SAT      164
2600     133
NES       97
GB        93
WiiU      72
DC        50
XOne      34
GEN       25
NG        10
Name: platform, dtype: int64
0


Replaced missing year of release with median year of release. Converted datatype to datetime.

In [641]:
print(ice_data['year_of_release'].isna().sum())
ice_data['year_of_release'] = ice_data['year_of_release'].where(ice_data['name'] != 'PES 2009: Pro Evolution Soccer', 2009)
ice_data['name_year'] = ice_data.query("year_of_release.isna() and (name.str.contains('200') or name.str.contains('19'))")['name'].str[-4:]
ice_data['year_of_release'] = ice_data['year_of_release'].where((ice_data['year_of_release'].notna() & ice_data['name_year'].isna()), ice_data['name_year'])
print(ice_data['year_of_release'].isna().sum())
year_median = ice_data['year_of_release'].median()
year_median = round(year_median, 0)
ice_data['year_of_release'] = ice_data['year_of_release'].where((ice_data['year_of_release'].notna()) , year_median) 
ice_data['year_of_release'] = ice_data['year_of_release'].astype(int)
ice_data['year_of_release'] = pd.to_datetime(ice_data['year_of_release'], format='%Y')
ice_data['year_of_release'] = ice_data['year_of_release'].dt.year
print(ice_data['year_of_release'].isna().sum())



232
220
0


This code checks for duplicates in the genre data by converting all the names to lowercase values and dropping any duplicates that also have a duplicate year of release.  

In [642]:
print(ice_data['genre'].value_counts())
print(ice_data['genre'].unique())
ice_data['genre_lowercase'] = ice_data['genre'].str.lower()
print(ice_data['genre_lowercase'].value_counts())
print(ice_data['genre_lowercase'].unique())

ice_data = ice_data.drop_duplicates(subset=['genre_lowercase', 'year_of_release']).reset_index(drop=True)

Action          2115
Sports          1484
Misc            1408
Role-Playing    1342
Adventure       1117
Shooter          906
Racing           849
Simulation       772
Fighting         662
Platform         633
Strategy         613
Puzzle           526
Name: genre, dtype: int64
['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan]
action          2115
sports          1484
misc            1408
role-playing    1342
adventure       1117
shooter          906
racing           849
simulation       772
fighting         662
platform         633
strategy         613
puzzle           526
Name: genre_lowercase, dtype: int64
['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy' nan]


This code checks for genres that are missing, then grabs the rows where the genre is missing to view for analysis.

It looks like this is the same row that was missing the name. Based on my previous analysis, I decided to keep the row and add the string "Unknown Genre" to the genre field.

In [643]:
print(ice_data['genre'].isna().sum())
print(ice_data.query("genre.isna()"))
ice_data['genre'] = ice_data['genre'].fillna('Unknown Genre')
print(ice_data['genre'].isna().sum())


1
             name platform  year_of_release genre  na_sales  eu_sales  \
235  Unknown Name      GEN             1993   NaN      1.78      0.53   

     jp_sales  other_sales  critic_score user_score rating name_lowercase  \
235       0.0         0.08           NaN        NaN    NaN            NaN   

    name_year genre_lowercase  
235       NaN             NaN  
0


This code calculates the mean critic score by year of release and fills in the missing critic score data with the appropriate mean by year.

In [644]:
print(ice_data['critic_score'].isna().sum())
critic_score_mean = ice_data.groupby('year_of_release')['critic_score'].mean()
critic_score_mean = critic_score_mean.fillna(0)
critic_score_mean = critic_score_mean.reset_index().rename(columns={0: 'year_of_release', 'critic_score': 'mean_critic_score'})
ice_data = ice_data.merge(critic_score_mean, on='year_of_release', how='left')
ice_data['critic_score'] = ice_data['critic_score'].fillna(ice_data['mean_critic_score'])
ice_data = ice_data.drop(columns='mean_critic_score')
print(ice_data['critic_score'].isna().sum())


214
0


This code examines the various possibilities for game ratings and then fills blank values with "Rating Unknown".

In [645]:
print(ice_data['rating'].isna().sum())
print(ice_data['rating'].unique())
ice_data['rating'] = ice_data['rating'].fillna('Rating Unknown')
print(ice_data['rating'].isna().sum())

208
['E' nan 'M' 'T' 'E10+' 'K-A']
0


This code calculates the mean user score by year of release and fills in the missing user score data with the appropriate mean by year.

In [646]:
print(ice_data['user_score'].isna().sum())
print(ice_data['user_score'].unique())
print(ice_data.query("user_score == 'tbd'"))
#ice_data['user_score'] = ice_data['user_score'].where((ice_data['user_score'] != 'tbd') , cylinder_median) 

ice_data['user_score'] = ice_data['user_score'].astype(float)
user_score_mean = ice_data.groupby('year_of_release')['user_score'].mean()
user_score_mean = user_score_mean.fillna(0)
user_score_mean = user_score_mean.reset_index().rename(columns={0: 'year_of_release', 'user_score': 'mean_user_score'})
ice_data = ice_data.merge(user_score_mean, on='year_of_release', how='left')
ice_data['user_score'] = ice_data['user_score'].fillna(ice_data['mean_user_score'])
ice_data = ice_data.drop(columns='mean_user_score')
print(ice_data['user_score'].isna().sum())

211
['8' nan '8.3' '8.5' '6.6' '8.4' '8.6' '7.7' '6.3' '8.2' '9' '7.9' '8.7'
 '7.1' '3.4' '5.3' '8.9' '7.8' '7.5' '2.6' '7.2' '9.2' '4.3' '7.6' '5.7'
 '5' '9.1' '6.5' '7.3' 'tbd' '8.8' '8.1' '6.8' '7.4' '6.9' '3.9' '6.2'
 '3.1' '5.5' '5.8' '2.2']
              name platform  year_of_release   genre  na_sales  eu_sales  \
93   Zumba Fitness      Wii             2010  Sports      3.45      2.59   
387           Lumo      PSV             2016  Puzzle      0.00      0.01   

     jp_sales  other_sales  critic_score user_score rating name_lowercase  \
93        0.0         0.66     84.400000        tbd      E  zumba fitness   
387       0.0         0.00     82.285714        tbd   E10+           lumo   

    name_year genre_lowercase  
93        NaN          sports  
387       NaN          puzzle  


ValueError: could not convert string to float: 'tbd'